# Functions

In [30]:
def compute_match_rates_and_sample_weights(match):
    a = max(0.001,min(match["p1 serves won"] / match["p1 serves total"],0.999))
    b = max(0.001,min(match["p2 serves won"] / match["p2 serves total"],0.999))
    c = max(0.001,min(match["p1 service games won"] / match["p1 service games total"],0.999))
    d = max(0.001,min(match["p2 service games won"] / match["p2 service games total"],0.999))
    e = max(0.001,min(match["p1 sets won"] / match["sets total"],0.999))
    f = float("nan")
    if match["p1 tiebreaks won"] != 0 or match["p2 tiebreaks won"] != 0:
        f = max(0.001,min(match["p1 tiebreaks won"] / (match["p1 tiebreaks won"] + match["p2 tiebreaks won"]),0.999))
        
    return a,b,c,d,e,f

def compute_sample_weights(match):
    a = match["p1 match serve win rate"] * (1-match["p1 match serve win rate"]) / match["p1 serves total"]
    b = match["p2 match serve win rate"] * (1-match["p2 match serve win rate"]) / match["p2 serves total"]
    c = match["p1 match service game win rate"] * (1-match["p1 match service game win rate"]) / match["p1 service games total"]
    d = match["p2 match service game win rate"] * (1-match["p2 match service game win rate"]) / match["p2 service games total"]
    e = match["p1 match set win rate"] * (1-match["p1 match set win rate"]) / match["sets total"]
    f = float("nan")
    if match["p1 tiebreaks won"] != 0 or match["p2 tiebreaks won"] != 0:
        f = match["p1 tiebreak win rate"] * (1-match["p1 tiebreak win rate"]) / (match["p1 tiebreaks won"] + match["p2 tiebreaks won"])
        
    return 1/a,1/b,1/c,1/d,1/e,1/f

In [31]:
def sigmoid(theta, x):
    return 1 / (1 + math.e**(-theta.dot(x)))

from sklearn.linear_model import LinearRegression


class LogitRegression(LinearRegression):
    def fit(self, x, p, sample_weight):
        # special care for 0 or 1 probabilities
        for i in range(len(p)):
            p[i] = max(0.001,min(p[i],0.999))
        p = np.asarray(p)
        y = np.log(p / (1 - p))
        return super().fit(x, y,sample_weight)

    def predict(self, x):
        y = super().predict(x)
        return 1 / (np.exp(-y) + 1)
#i = 0
def compute_economist_rates(match, matches):
    #global i
    # if either player has played less than two matches in the past year, return nan
    if match["p1 match count"] < 2 or match["p2 match count"] < 2:
        return float("nan"),float("nan")
    
    date = match["date"]
    one_year_prior = date - relativedelta(years=1)
    relevant_matches = matches[(matches["date"] < date) & (matches["date"] >= one_year_prior)]
    
    # find players in past year
    players = set()
    for index, match_ in relevant_matches.iterrows():
        players.add(match_["server1"])
        players.add(match_["server2"])
    players = list(players)
    players_dict = {}
    for i in range(len(players)):
        players_dict[players[i]] = i
    
    
    # construct features for p1 serves to p2
    features = []
    for index, match_ in relevant_matches.iterrows():
        # first are the serve abilities, next the return abilities
        feature = [0 for i in range(2*len(players))]
        feature[players_dict[match_["server1"]]] = 1
        feature[len(players) + players_dict[match_["server2"]]] = 1
        features.append(feature)
        
    # construct features for p2 serves to p1
    for index, match_ in relevant_matches.iterrows():
        # first are the serve abilities, next the return abilities
        feature = [0 for i in range(2*len(players))]
        feature[players_dict[match_["server2"]]] = 1
        feature[len(players) + players_dict[match_["server1"]]] = 1
        features.append(feature)
    
    # logistic regression
    features = np.array(features)
    w1 = np.array(relevant_matches["p1 match inv serve var"])
    w2 = np.array(relevant_matches["p2 match inv serve var"])
    weights = np.concatenate([w1,w2])
    outcome_col = np.append(relevant_matches["p1 match serve win rate"].to_numpy(), relevant_matches["p2 match serve win rate"].to_numpy())
    reg = LogitRegression()
    reg.fit(features,outcome_col,weights)
    
    # compute estimated serve win rate for match from p1 to p2
    p1_to_p2 = [0 for i in range(2*len(players))]
    p1_to_p2[players_dict[match["server1"]]] = 1
    p1_to_p2[len(players) + players_dict[match["server2"]]] = 1
    p1_to_p2 = np.array([p1_to_p2])
    a = reg.predict(p1_to_p2)
    
    # compute estimated serve win rate for match from p2 to p1
    p2_to_p1 = [0 for i in range(2*len(players))]
    p2_to_p1[players_dict[match["server2"]]] = 1
    p2_to_p1[len(players) + players_dict[match["server1"]]] = 1
    p2_to_p1 = np.array([p2_to_p1])
    b = reg.predict(p2_to_p1)
    #print(i)
    #i += 1
    return a[0],b[0]
    
    
    
    
    

In [32]:
count = [0,0,0,0]
def compute_economist_sg_rg_rates(match, matches):
    global count
    # if either player has played less than two matches in the past year, return nan
    if match["p1 match count"] < 2 or match["p2 match count"] < 2:
        return float("nan"),float("nan")
    
    date = match["date"]
    one_year_prior = date - relativedelta(years=1)
    relevant_matches = matches[(matches["date"] < date) & (matches["date"] >= one_year_prior)]
    
    # find players in past year
    players = set()
    for index, match_ in relevant_matches.iterrows():
        players.add(match_["server1"])
        players.add(match_["server2"])
    players = list(players)
    players_dict = {}
    for i in range(len(players)):
        players_dict[players[i]] = i
    
    
    # construct features for p1 serves to p2
    features = []
    for index, match_ in relevant_matches.iterrows():
        # first are the serve abilities, next the return abilities
        feature = [0 for i in range(2*len(players))]
        feature[players_dict[match_["server1"]]] = 1
        feature[len(players) + players_dict[match_["server2"]]] = 1
        features.append(feature)
        
    # construct features for p2 serves to p1
    for index, match_ in relevant_matches.iterrows():
        # first are the serve abilities, next the return abilities
        feature = [0 for i in range(2*len(players))]
        feature[players_dict[match_["server2"]]] = 1
        feature[len(players) + players_dict[match_["server1"]]] = 1
        features.append(feature)
    
    # logistic regression
    features = np.array(features)
    w1 = np.array(relevant_matches["p1 match inv service game var"])
    w2 = np.array(relevant_matches["p2 match inv service game var"])
    weights = np.concatenate([w1,w2])
    outcome_col = np.append(relevant_matches["p1 match service game win rate"].to_numpy(), relevant_matches["p2 match service game win rate"].to_numpy())
    reg = LogitRegression()
    reg.fit(features,outcome_col,weights)
    
    # compute estimated serve win rate for match from p1 to p2
    p1_to_p2 = [0 for i in range(2*len(players))]
    p1_to_p2[players_dict[match["server1"]]] = 1
    p1_to_p2[len(players) + players_dict[match["server2"]]] = 1
    p1_to_p2 = np.array([p1_to_p2])
    a = reg.predict(p1_to_p2)
    
    # compute estimated serve win rate for match from p2 to p1
    p2_to_p1 = [0 for i in range(2*len(players))]
    p2_to_p1[players_dict[match["server2"]]] = 1
    p2_to_p1[len(players) + players_dict[match["server1"]]] = 1
    p2_to_p1 = np.array([p2_to_p1])
    b = reg.predict(p2_to_p1)
    
    if match["pbp_id"] == 8634739:
        print(reg.coef_[players_dict[match["server1"]]])
        print(reg.coef_[players_dict[match["server2"]]])
        print(reg.coef_[len(players) + players_dict[match["server1"]]])
        print(reg.coef_[len(players) + players_dict[match["server2"]]])
        
    if reg.coef_[players_dict[match["server1"]]] > 0:
        count[0] += 1
    else:
        count[1] += 1
        
    if reg.coef_[players_dict[match["server2"]]] > 0:
        count[0] += 1
    else:
        count[1] += 1
        
    if reg.coef_[len(players) + players_dict[match["server1"]]] > 0:
        count[2] += 1
    else:
        count[3] += 1
        
    if reg.coef_[len(players) + players_dict[match["server1"]]] > 0:
        count[2] += 1
    else:
        count[3] += 1
    
    

    return a[0],b[0]
    
    
    
    
    

# Code

In [33]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
from scipy.stats import beta
from multiprocessing import Pool, freeze_support
import math
from sklearn import linear_model
pd.set_option("display.max_columns", None)

In [34]:
matches = pd.read_csv("./data/2011-2015_matches_cleaned_forShrinkage.csv")
matches["date"] = matches.apply(lambda x: datetime.strptime(x["date"], '%Y-%m-%d'), axis=1)
matches

,pbp_id,date,tny_name,tour,draw,server1,server2,winner,pbp,score,adf_flag,wh_minutes,p1 serves won,p1 serves total,p1 service games won,p1 service games total,p1 sets won,p1 tiebreaks won,p2 serves won,p2 serves total,p2 service games won,p2 service games total,p2 sets won,p2 tiebreaks won,sets total,tourney serves won,tourney serves total,tourney service games won,tourney service games total,p1 serve win rate,p1 return win rate,p1 service game win rate,p1 return game win rate,p1 set win rate,p1 tiebreak win rate,p2 serve win rate,p2 return win rate,p2 service game win rate,p2 return game win rate,p2 set win rate,p2 tiebreak win rate,p1 opp serve win rate,p1 opp return win rate,p1 opp service game win rate,p1 opp return game win rate,p1 opp set win rate,p1 opp tiebreak win rate,p2 opp serve win rate,p2 opp return win rate,p2 opp service game win rate,p2 opp return game win rate,p2 opp set win rate,p2 opp tiebreak win rate,p1 match count,p2 match count,all player year serve win rate,all player year service game win rate,p1 year serve total,p1 year return total,p1 year service game total,p1 year return game total,p1 year set total,p1 year tiebreak total,p2 year serve total,p2 year return total,p2 year service game total,p2 year return game total,p2 year set total,p2 year tiebreak total,p1 year match win rate,p2 year match rate,p1 year match results,p2 year match results,p1 opp match win rate,p2 opp match win rate,p1 relative serve,p2 relative serve,p1 relative return,p2 relative return,p1 relative service games,p2 relative service games,p1 relative return games,p2 relative return games,p1 relative set,p2 relative set,p1 relative tiebreak,p2 relative tiebreak,tourney serve win rate,tourney return win rate,tourney service game win rate,tourney return game win rate,p1 tourney serve adv,p1 tourney return adv,p2 tourney serve adv,p2 tourney return adv,p1 tourney service game adv,p1 tourney return game adv,p2 tourney service game adv,p2 tourney return game adv,p1 tourney set adv,p2 tourney set adv,p1 tourney tiebreak adv,p2 tourney tiebreak adv,tourney sets to win,p1 serve_tt,p1 service_game_tt,p1 set_tt,p1 tiebreak_tt,p2 serve_tt,p2 service_game_tt,p1 serve_tt no sos adjust,p2 serve_tt no sos adjust,p1 service_game_tt no sos adjust,p2 service_game_tt no sos adjust
0,2231275,2011-07-28,ATPStu,ATP,Main,Olivier Rochus,Fabio Fognini,2,SSSS;RRRR;SSRRSS;SSRRSS;RSRSRSRR;SSRSS;RSRRSR;...,6-4 6-1,0,66,27,58,3,9,0,0,32,56,6,8,2,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0,2,2,2,2,0,0,2,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2231276,2011-07-28,ATPStu,ATP,Main,Robin Haase,Marin Cilic,2,SSRSS;RRSSRSSS;SSSS;RSSSS;SRSRSS;RSRSRSSS;RSRS...,4-6 6-4 6-3,0,141,60,102,11,15,1,0,60,92,12,14,2,0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0,2,2,2,2,0,0,2,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2231342,2011-07-28,Farmer,ATP,Main,Alejandro Falla,Thomaz Bellucci,2,SRRSSRSRRSRSRSRSRSRR;SSSRS;RRSRR;SRSSS;RRSSRR;...,6-0 6-1,0,42,21,55,1,7,0,0,26,37,6,6,2,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0,2,2,2,2,0,0,2,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2229262,2011-07-28,Credit,ATP,Main,Matthias Bachinger,Julien Benneteau,2,SSSS;SSSS;RRRR;SSSS;SSSS;SRRRSSRSSRRSSRSS;SSSS...,6-4 6-4,0,57,36,59,7,10,0,0,43,58,9,10,2,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [35]:
match_rates = ["p1 match serve win rate", "p2 match serve win rate", "p1 match service game win rate", "p2 match service game win rate", "p1 match set win rate", "p1 match tiebreak win rate"]
matches[match_rates] = matches.apply(lambda match: compute_match_rates(match), axis=1, result_type="expand")

match_var_rates = ["p1 match inv serve var", "p2 match inv serve var", "p1 match inv service game var", "p2 match inv service game var", "p1 match inv set var", "p1 match inv tiebreak var"]
matches[match_var_rates] = matches.apply(lambda match: compute_match_var_rates(match), axis=1, result_type="expand")




In [36]:
economist_serve_rates = ["p1 serve_tt economist", "p2 serve_tt economist"]
matches[economist_serve_rates] = matches.apply(lambda match: compute_economist_rates(match, matches), axis=1, result_type="expand")


matches

/Users/VillasBoas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: overflow encountered in exp
/Users/VillasBoas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: overflow encountered in exp
/Users/VillasBoas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: overflow encountered in exp
/Users/VillasBoas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: overflow encountered in exp
/Users/VillasBoas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: overflow encountered in exp
/Users/VillasBoas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: overflow encountered in exp
/Users/VillasBoas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: overflow encountered in exp
/Users/VillasBoas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: overflow encountered

,pbp_id,date,tny_name,tour,draw,server1,server2,winner,pbp,score,adf_flag,wh_minutes,p1 serves won,p1 serves total,p1 service games won,p1 service games total,p1 sets won,p1 tiebreaks won,p2 serves won,p2 serves total,p2 service games won,p2 service games total,p2 sets won,p2 tiebreaks won,sets total,tourney serves won,tourney serves total,tourney service games won,tourney service games total,p1 serve win rate,p1 return win rate,p1 service game win rate,p1 return game win rate,p1 set win rate,p1 tiebreak win rate,p2 serve win rate,p2 return win rate,p2 service game win rate,p2 return game win rate,p2 set win rate,p2 tiebreak win rate,p1 opp serve win rate,p1 opp return win rate,p1 opp service game win rate,p1 opp return game win rate,p1 opp set win rate,p1 opp tiebreak win rate,p2 opp serve win rate,p2 opp return win rate,p2 opp service game win rate,p2 opp return game win rate,p2 opp set win rate,p2 opp tiebreak win rate,p1 match count,p2 match count,all player year serve win rate,all player year service game win rate,p1 year serve total,p1 year return total,p1 year service game total,p1 year return game total,p1 year set total,p1 year tiebreak total,p2 year serve total,p2 year return total,p2 year service game total,p2 year return game total,p2 year set total,p2 year tiebreak total,p1 year match win rate,p2 year match rate,p1 year match results,p2 year match results,p1 opp match win rate,p2 opp match win rate,p1 relative serve,p2 relative serve,p1 relative return,p2 relative return,p1 relative service games,p2 relative service games,p1 relative return games,p2 relative return games,p1 relative set,p2 relative set,p1 relative tiebreak,p2 relative tiebreak,tourney serve win rate,tourney return win rate,tourney service game win rate,tourney return game win rate,p1 tourney serve adv,p1 tourney return adv,p2 tourney serve adv,p2 tourney return adv,p1 tourney service game adv,p1 tourney return game adv,p2 tourney service game adv,p2 tourney return game adv,p1 tourney set adv,p2 tourney set adv,p1 tourney tiebreak adv,p2 tourney tiebreak adv,tourney sets to win,p1 serve_tt,p1 service_game_tt,p1 set_tt,p1 tiebreak_tt,p2 serve_tt,p2 service_game_tt,p1 serve_tt no sos adjust,p2 serve_tt no sos adjust,p1 service_game_tt no sos adjust,p2 service_game_tt no sos adjust,p1 match serve win rate,p2 match serve win rate,p1 match service game win rate,p2 match service game win rate,p1 match set win rate,p1 match tiebreak win rate,p1 match inv serve var,p2 match inv serve var,p1 match inv service game var,p2 match inv service game var,p1 match inv set var,p1 match inv tiebreak var,p1 serve_tt economist,p2 serve_tt economist
0,2231275,2011-07-28,ATPStu,ATP,Main,Olivier Rochus,Fabio Fognini,2,SSSS;RRRR;SSRRSS;SSRRSS;RSRSRSRR;SSRSS;RSRRSR;...,6-4 6-1,0,66,27,58,3,9,0,0,32,56,6,8,2,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0,2,2,2,2,0,0,2,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.465517,0.571429,0.333333,0.750000,0.000000,NaN,0.004290,0.004373,0.024691,0.023438,0.000000,NaN,NaN,NaN
1,2231276,2011-07-28,ATPStu,ATP,Main,Robin Haase,Marin Cilic,2,SSRSS;RRSSRSSS;SSSS;RSSSS;SRSRSS;RSRSRSSS;RSRS...,4-6 6-4 6-3,0,141,60,102,11,15,1,0,60,92,12,14,2,0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0,2,2,2,2,0,0,2,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.588235,0.652174,0.733333,0.857143,0.333333,NaN,0.002375,0.002466,0.013037,0.008746,0.074074,NaN,NaN,NaN
2,2231342,2011-07-28,Farmer,ATP,Main,Alejandro Falla,Thomaz Bellucci,2,SRRSSRSRRSRSRSRSRSRR;SSSRS;RRSRR;SRSSS;RRSSRR;...,6-0 6-1,0,42,21,55,1,7,0,0,26,37,6

In [37]:
economist_serve_rates = ["p1 service_game_tt economist", "p2 service_game_tt economist"]
matches[economist_serve_rates] = matches.apply(lambda match: compute_economist_sg_rg_rates(match, matches), axis=1, result_type="expand")


matches

/Users/VillasBoas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: overflow encountered in exp
/Users/VillasBoas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: overflow encountered in exp
/Users/VillasBoas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: overflow encountered in exp
/Users/VillasBoas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: overflow encountered in exp
/Users/VillasBoas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: overflow encountered in exp
/Users/VillasBoas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: overflow encountered in exp
/Users/VillasBoas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: overflow encountered in exp
/Users/VillasBoas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: overflow encountered

-12186782539485.066
-12186782539486.035
283147765926.1382
283147765927.6019


,pbp_id,date,tny_name,tour,draw,server1,server2,winner,pbp,score,adf_flag,wh_minutes,p1 serves won,p1 serves total,p1 service games won,p1 service games total,p1 sets won,p1 tiebreaks won,p2 serves won,p2 serves total,p2 service games won,p2 service games total,p2 sets won,p2 tiebreaks won,sets total,tourney serves won,tourney serves total,tourney service games won,tourney service games total,p1 serve win rate,p1 return win rate,p1 service game win rate,p1 return game win rate,p1 set win rate,p1 tiebreak win rate,p2 serve win rate,p2 return win rate,p2 service game win rate,p2 return game win rate,p2 set win rate,p2 tiebreak win rate,p1 opp serve win rate,p1 opp return win rate,p1 opp service game win rate,p1 opp return game win rate,p1 opp set win rate,p1 opp tiebreak win rate,p2 opp serve win rate,p2 opp return win rate,p2 opp service game win rate,p2 opp return game win rate,p2 opp set win rate,p2 opp tiebreak win rate,p1 match count,p2 match count,all player year serve win rate,all player year service game win rate,p1 year serve total,p1 year return total,p1 year service game total,p1 year return game total,p1 year set total,p1 year tiebreak total,p2 year serve total,p2 year return total,p2 year service game total,p2 year return game total,p2 year set total,p2 year tiebreak total,p1 year match win rate,p2 year match rate,p1 year match results,p2 year match results,p1 opp match win rate,p2 opp match win rate,p1 relative serve,p2 relative serve,p1 relative return,p2 relative return,p1 relative service games,p2 relative service games,p1 relative return games,p2 relative return games,p1 relative set,p2 relative set,p1 relative tiebreak,p2 relative tiebreak,tourney serve win rate,tourney return win rate,tourney service game win rate,tourney return game win rate,p1 tourney serve adv,p1 tourney return adv,p2 tourney serve adv,p2 tourney return adv,p1 tourney service game adv,p1 tourney return game adv,p2 tourney service game adv,p2 tourney return game adv,p1 tourney set adv,p2 tourney set adv,p1 tourney tiebreak adv,p2 tourney tiebreak adv,tourney sets to win,p1 serve_tt,p1 service_game_tt,p1 set_tt,p1 tiebreak_tt,p2 serve_tt,p2 service_game_tt,p1 serve_tt no sos adjust,p2 serve_tt no sos adjust,p1 service_game_tt no sos adjust,p2 service_game_tt no sos adjust,p1 match serve win rate,p2 match serve win rate,p1 match service game win rate,p2 match service game win rate,p1 match set win rate,p1 match tiebreak win rate,p1 match inv serve var,p2 match inv serve var,p1 match inv service game var,p2 match inv service game var,p1 match inv set var,p1 match inv tiebreak var,p1 serve_tt economist,p2 serve_tt economist,p1 service_game_tt economist,p2 service_game_tt economist
0,2231275,2011-07-28,ATPStu,ATP,Main,Olivier Rochus,Fabio Fognini,2,SSSS;RRRR;SSRRSS;SSRRSS;RSRSRSRR;SSRSS;RSRRSR;...,6-4 6-1,0,66,27,58,3,9,0,0,32,56,6,8,2,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0,2,2,2,2,0,0,2,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.465517,0.571429,0.333333,0.750000,0.000000,NaN,0.004290,0.004373,0.024691,0.023438,0.000000,NaN,NaN,NaN,NaN,NaN
1,2231276,2011-07-28,ATPStu,ATP,Main,Robin Haase,Marin Cilic,2,SSRSS;RRSSRSSS;SSSS;RSSSS;SRSRSS;RSRSRSSS;RSRS...,4-6 6-4 6-3,0,141,60,102,11,15,1,0,60,92,12,14,2,0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0,2,2,2,2,0,0,2,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.588235,0.652174,0.733333,0.857143,0.333333,NaN,0.002375,0.002466,0.013037,0.008746,0.074074,NaN,NaN,NaN,NaN,NaN
2,2231342,2011-07-28,Farmer,ATP,Main,Alejandro Falla,Thomaz Bellucci,2,SRRSSRSRRS

In [14]:
x = np.array([1,2,3])
y = np.array([3,4,5])
np.append(x,y)

array([1, 2, 3, 3, 4, 5])

In [43]:
features

NameError: name 'features' is not defined

In [47]:
matches[match_rates].describe()

,p1 match serve win rate,p2 match serve win rate,p1 match service game win rate,p2 match service game win rate,p1 match set win rate,p1 match tiebreak win rate
count,10787.000000,10787.000000,10787.000000,10787.000000,10787.000000,4047.000000
mean,0.641515,0.632156,0.787912,0.771705,0.521623,0.514496
std,0.091863,0.091779,0.172940,0.177159,0.410931,0.477276
min,0.000000,0.241379,0.000000,0.000000,0.000000,0.000000
25%,0.581633,0.574074,0.692308,0.666667,0.000000,0.000000
50%,0.644737,0.633333,0.812500,0.800000,0.666667,0.500000
75%,0.704762,0.693954,0.909091,0.900000,1.000000,1.000000
max,0.941176,0.947368,1.000000,1.000000,1.000000,1.000000


# Try to predict on test matches

### predict using serve win rates

In [38]:
# Filter matches we want to predict on

# Only consider matches on or after August 1, 2012, because we've had one year of data at that point
games_after_2012 = matches[matches["date"] > datetime.strptime("31 Dec 14", '%d %b %y')]

# Only consider best of three set matches
games_after_2012 = games_after_2012[games_after_2012["tourney sets to win"] == 2]

# Don't consider Davis Cup matches because they don't have a final set tiebreak
games_after_2012 = games_after_2012[games_after_2012["tny_name"] != "DavisC"]

# Remove rows where either player hasn't played at least 10 matches in the past year
games_after_2012 = games_after_2012[(games_after_2012["p1 match count"] >= 2) & (games_after_2012["p2 match count"] >= 2)]

In [39]:
# Remove rows where we don't have s_ijtt values to predict the result of the match
games_after_2012 = games_after_2012[(games_after_2012['p1 serve_tt economist'].notna()) & (games_after_2012['p2 serve_tt economist'].notna())]





In [40]:
# import the markov model functions lazily
%run Modeling_tennis_match_iid_points.ipynb

0.69875
0.933699


In [41]:
# compute Pr[p1 wins] with opponent/ tournament adjusted shrunken serve values
games_after_2012["p1 pr win"] = games_after_2012.apply(lambda match: match_driver(2, 6, 4, 7, 1, match["p1 serve_tt economist"], match["p2 serve_tt economist"]), axis=1).clip(upper=0.9985, lower=0.0015)




In [42]:
# compute log loss
games_after_2012["winner"] = games_after_2012["winner"].replace(2,0)

compute_lloss = games_after_2012[["winner", "p1 pr win"]]

compute_lloss["lloss"] = compute_lloss.apply(lambda m: (m["winner"]*np.log(m["p1 pr win"]) + (1-m["winner"])*np.log(1 - m["p1 pr win"])), axis=1)
N = compute_lloss.shape[0]

lloss = (compute_lloss["lloss"].sum()) * (-1/N)

print(f"log loss: {lloss}")
print(f"Random guessing log loss: {-np.log(0.5)}")

log loss: 0.6775073990470023
Random guessing log loss: 0.6931471805599453


/Users/VillasBoas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### predict using sg,rg and serve for tiebreak

In [43]:
# Filter matches we want to predict on

# Only consider matches on or after August 1, 2012, because we've had one year of data at that point
games_after_2012 = matches[matches["date"] > datetime.strptime("31 Dec 14", '%d %b %y')]

# Only consider best of three set matches
games_after_2012 = games_after_2012[games_after_2012["tourney sets to win"] == 2]

# Don't consider Davis Cup matches because they don't have a final set tiebreak
games_after_2012 = games_after_2012[games_after_2012["tny_name"] != "DavisC"]

# Remove rows where either player hasn't played at least 10 matches in the past year
games_after_2012 = games_after_2012[(games_after_2012["p1 match count"] >= 2) & (games_after_2012["p2 match count"] >= 2)]



In [44]:
# Remove rows where we don't have sg_ijtt and s_ijtt values to predict the result of the match
games_after_2012 = games_after_2012[(games_after_2012['p1 service_game_tt economist'].notna()) & (games_after_2012['p2 service_game_tt economist'].notna()) & (games_after_2012['p1 serve_tt economist'].notna()) & (games_after_2012['p2 serve_tt economist'].notna())]





In [45]:
# compute Pr[p1 wins] with non opponenent/ tournament adjusted service game values
games_after_2012["p1 pr win"] = games_after_2012.apply(lambda match: match_driver_game_based_tiebreak_serve_based(2, 6, 4, 7, 1, match["p1 service_game_tt economist"], match["p2 service_game_tt economist"], match["p1 serve_tt economist"], match["p2 serve_tt economist"]), axis=1).clip(upper=0.9985, lower=0.0015)






In [46]:
games_after_2012["winner"] = games_after_2012["winner"].replace(2,0)

compute_lloss = games_after_2012[["winner", "p1 pr win"]]

compute_lloss["lloss"] = compute_lloss.apply(lambda m: (m["winner"]*np.log(m["p1 pr win"]) + (1-m["winner"])*np.log(1 - m["p1 pr win"])), axis=1)
N = compute_lloss.shape[0]

lloss = (compute_lloss["lloss"].sum()) * (-1/N)

print(f"log loss: {lloss}")
print(f"Random guessing log loss: {-np.log(0.5)}")

log loss: 0.6525481363919573
Random guessing log loss: 0.6931471805599453


/Users/VillasBoas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [98]:
count

[9624, 9462, 9468, 9618]